In [96]:
import numpy as np
import pandas as pd
import os

In [104]:
path = 'Final/'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))

In [105]:
files[0]

'Final/927_0196_01.csv'

In [106]:
descriptors = pd.read_csv('Descriptors.csv', index_col=0)
descriptors.head()

,gender,gender_no,output,expression_index,video_name,start_time_in_sec,end_time_in_sec,start_exp,end_exp,text_file,kinect_start_frame,kinect_end_frame
1,F,3,An,1,927_0216_01.MP4,5.356700,9.481139,161,284,20170902_092359_00.txt,217,312
2,F,3,An,2,927_0216_01.MP4,9.883585,14.903150,296,447,20170902_092359_00.txt,322,466
3,F,3,An,3,927_0216_01.MP4,14.903150,19.427206,447,582,20170902_092359_00.txt,466,564
4,F,3,An,4,927_0216_01.MP4,19.427206,25.077029,582,752,20170902_092359_00.txt,564,699
5,F,3,An,5,927_0216_01.MP4,25.077029,29.981168,752,899,20170902_092359_00.txt,699,828


In [114]:
def load_file(file):
    temp_df = pd.read_csv(file, sep=',')
    max_value = np.amax(temp_df)
    max_value = max(max_value)
    temp_df = temp_df/max_value
    #print(max_value)
    name = file[6:-4]
    output_value = descriptors.loc[descriptors['video_name']==name +'.MP4']['output'][:1].values[0]
    #video_name = descriptors.loc[descriptors['text_file']==textfile]['video_name'][:1].values[0]
    gender = descriptors.loc[descriptors['video_name']==name +'.MP4']['gender'][:1].values[0]
    #temp_df['textfile'] = textfile
    #temp_df['output'] = output_value
    #temp_df['video_name'] = video_name
    if gender == 'F':
        temp_df['gender'] = 1
    else:
        temp_df['gender'] = 0
    #temp_df.drop(['Unnamed: 152'], axis=1, inplace=True)
    return temp_df.values, output_value

In [115]:
a, b = load_file('Final/927_0196_01.csv')

In [129]:
a.shape

(791, 2279)

In [117]:
shapes = []
Y = []
for name in files:
    data, t = load_file(name)
    shapes.append(data.shape[0])
    Y.append(t)

In [119]:
index = 0 
for element in Y:
    if element == 'Ne':
        Y[index] = 0
    elif element == 'Sa':
        Y[index] = 1
    elif element == 'Su':
        Y[index] = 2
    elif element == 'Fe':
        Y[index] = 3
    elif element == 'An':
        Y[index] = 4
    elif element == 'Di':
        Y[index] = 5
    elif element == 'Ha':
        Y[index] = 6
    index += 1

In [120]:
from tensorflow.keras.utils import to_categorical

maxlen = max(shapes)
print(maxlen)

1864


In [132]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data, temp = load_file(name)
        #print('The name is:', name, data.shape)
        zeros_data = np.zeros((max(shapes)-data.shape[0], 2279))
        print(data.shape)
        data = np.concatenate((zeros_data, data), axis=0)
        #print('Now the shape is:', name, data.shape)
        loaded.append(data)
        shapes.append(data.shape[0])
    loaded = np.stack(loaded)
    return loaded

In [133]:
X = load_group(files)

Y = np.asarray(Y).reshape((63,1))
Y = to_categorical(Y)
print(X.shape, Y.shape)

(791, 2279)
(1067, 2279)
(1112, 2279)
(1362, 2279)
(965, 2279)
(1252, 2279)
(1027, 2279)
(1035, 2279)
(1419, 2279)
(1321, 2279)
(1351, 2279)
(952, 2279)
(1310, 2279)
(907, 2279)
(914, 2279)
(1131, 2279)
(972, 2279)
(919, 2279)
(1083, 2279)
(1046, 2279)
(1022, 2279)
(842, 2279)
(950, 2279)
(1265, 2279)
(796, 2279)
(824, 2279)
(781, 2279)
(738, 2279)
(895, 2279)
(1124, 2279)
(1021, 2279)
(1174, 2279)
(852, 2279)
(869, 2279)
(723, 2279)
(792, 2279)
(758, 2279)
(775, 2279)
(742, 2279)
(742, 2279)
(1605, 2279)
(575, 2279)
(836, 2279)
(1147, 2279)
(1105, 2279)
(1174, 2279)
(733, 2279)
(1165, 2279)
(682, 2279)
(1060, 2279)
(1097, 2279)
(949, 2279)
(1295, 2279)
(841, 2279)
(1182, 2279)
(958, 2279)
(1864, 2279)
(1577, 2279)
(1150, 2279)
(1224, 2279)
(887, 2279)
(1332, 2279)
(1055, 2279)
(63, 1864, 2279) (63, 7)


In [134]:
np.save('X.npy', X)
np.save('Y.npy', Y)

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)

In [136]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [137]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose = 1
    epochs = 100
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(32, input_shape=(n_timesteps, n_features)))
    #model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.fit(trainX, trainy, epochs=epochs, verbose=verbose)
    _, accuracy = model.evaluate(testX, testy, verbose=verbose)
    return accuracy

In [138]:
evaluate_model(X_train, Y_train, X_test, Y_test)

Epoch 1/100
56/56 [==============================] - 36s 637ms/sample - loss: 2.0371 - accuracy: 0.1071
Epoch 2/100
56/56 [==============================] - 16s 278ms/sample - loss: 1.9467 - accuracy: 0.1429
Epoch 3/100
56/56 [==============================] - 12s 212ms/sample - loss: 1.9439 - accuracy: 0.1607
Epoch 4/100
56/56 [==============================] - 10s 183ms/sample - loss: 1.9432 - accuracy: 0.1607
Epoch 5/100
56/56 [==============================] - 10s 179ms/sample - loss: 1.9437 - accuracy: 0.1607
Epoch 6/100
56/56 [==============================] - 10s 175ms/sample - loss: 1.9420 - accuracy: 0.1607
Epoch 7/100
56/56 [==============================] - 10s 176ms/sample - loss: 1.9416 - accuracy: 0.1607
Epoch 8/100
56/56 [==============================] - 10s 177ms/sample - loss: 1.9420 - accuracy: 0.1607
Epoch 9/100
56/56 [==============================] - 10s 175ms/sample - loss: 1.9413 - accuracy: 0.1607
Epoch 10/100
56/56 [==============================] - 10s 176ms/

56/56 [==============================] - 21s 366ms/sample - loss: 1.8688 - accuracy: 0.3393
Epoch 80/100
56/56 [==============================] - 20s 360ms/sample - loss: 1.8669 - accuracy: 0.3214
Epoch 81/100
56/56 [==============================] - 20s 361ms/sample - loss: 1.8627 - accuracy: 0.3214
Epoch 82/100
56/56 [==============================] - 20s 360ms/sample - loss: 1.8558 - accuracy: 0.3750
Epoch 83/100
56/56 [==============================] - 21s 368ms/sample - loss: 1.8526 - accuracy: 0.3036
Epoch 84/100
56/56 [==============================] - 20s 362ms/sample - loss: 1.8584 - accuracy: 0.3214
Epoch 85/100
56/56 [==============================] - 20s 359ms/sample - loss: 1.8452 - accuracy: 0.3214
Epoch 86/100
56/56 [==============================] - 20s 361ms/sample - loss: 1.8507 - accuracy: 0.2679
Epoch 87/100
56/56 [==============================] - 20s 361ms/sample - loss: 1.8468 - accuracy: 0.3393
Epoch 88/100
56/56 [==============================] - 20s 366ms/samp

0.0